## LangGraph Agents Tutorial

This notebook demonstrates how to build AI agents using LangGraph, OpenAI GPT models, and external tools. Agents are AI systems that can reason about problems, decide when to use tools, and execute multi-step workflows autonomously.

### What You'll Learn:
- How to create and configure tools for agents
- Tool binding - connecting LLMs with external capabilities  
- Building ReAct agents using LangGraph's prebuilt patterns
- Agent execution and message handling

## Environment Setup

Install required packages for building LangGraph agents. Each package serves a specific purpose:
- **langchain**: Core framework for building LLM applications
- **langgraph**: Graph-based orchestration for complex agent workflows  
- **openai & langchain_openai**: Integration with OpenAI's GPT models
- **wikipedia**: External tool for information retrieval

In [ ]:
# !pip install -q langchain langchain-community langgraph openai langchain_openai wikipedia

## Tool Creation: Wikipedia Search

**Tools** are external capabilities that agents can use to gather information or perform actions. Here we create a Wikipedia search tool:

- **WikipediaAPIWrapper**: Handles API communication with Wikipedia
- **top_k_results=1**: Limits results to prevent information overload
- **doc_content_chars_max=300**: Restricts content length for efficient processing
- **WikipediaQueryRun**: LangChain tool wrapper that makes the API accessible to agents

In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=300)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

## Testing the Tool Directly

Before integrating tools with agents, it's good practice to test them independently to ensure they work correctly and return expected results.

In [2]:
wiki_tool.run({"query": "AI agents"})

'Page: Agentic AI\nSummary: Agentic AI is a class of artificial intelligence that focuses on autonomous systems that can make decisions and perform tasks without human intervention. The independent systems automatically respond to conditions, to produce process results. The field is closely linked to '

## LLM Configuration

Set up the Language Model that will power our agent's reasoning capabilities:

- **ChatOpenAI**: LangChain's interface to OpenAI's models
- **temperature=0**: Ensures deterministic, consistent responses
- **gpt-4o-mini**: Cost-effective model suitable for most agent tasks
- **API key**: Required for authentication (use environment variables in production)

In [ ]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

llm = ChatOpenAI(temperature=0.3, model="gpt-4.1-nano")

## Tool Binding: Connecting LLM with Tools

**Tool binding** makes the LLM aware of available tools and enables it to generate tool calls when needed:

1. **bind_tools()**: Registers tools with the LLM, providing their descriptions and schemas
2. **Tool calling**: The LLM can now decide when and how to use tools based on user queries
3. The LLM will respond with either direct text or tool call instructions

In [4]:
tools = [wiki_tool]

# Tool binding
llm_with_tools = llm.bind_tools(tools)

#Tool calling
result = llm_with_tools.invoke("Hello world!")
result.content

'Hello! How can I assist you today?'

## ReAct Agent Creation

**ReAct (Reasoning + Acting)** is a proven agent pattern that combines:

- **Reasoning**: LLM thinks through the problem step-by-step
- **Acting**: LLM decides which tools to use and when
- **Observation**: LLM processes tool results and continues reasoning

`create_react_agent()` provides a prebuilt implementation that handles the ReAct loop automatically.

In [5]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

## Testing Agent Response (No Tools)

Test how the agent handles simple queries that don't require external tools. The agent should respond directly without calling Wikipedia since it can answer "hi!" from its training data.

In [6]:
from langchain_core.messages import HumanMessage

#First up, let's see how it responds when there's no need to call a tool:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='0b0a0574-313e-45be-b70f-5fef39ecef6f'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_c4c155951e', 'id': 'chatcmpl-C9j8lM06sLtBzTF4RPGsoALTpa1H5', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--20e6e7aa-bbc9-4b56-b325-8083d1442edc-0', usage_metadata={'input_tokens': 83, 'output_tokens': 10, 'total_tokens': 93, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

In [7]:
import pandas as pd
pd.DataFrame([msg.__dict__ for msg in response["messages"]])

,content,additional_kwargs,response_metadata,type,name,id,example,tool_calls,invalid_tool_calls,usage_metadata
0,hi!,{},{},human,None,0b0a0574-313e-45be-b70f-5fef39ecef6f,False,NaN,NaN,NaN
1,Hello! How can I assist you today?,{'refusal': None},"{'token_usage': {'completion_tokens': 10, 'pro...",ai,None,run--20e6e7aa-bbc9-4b56-b325-8083d1442edc-0,False,[],[],"{'input_tokens': 83, 'output_tokens': 10, 'tot..."


In [8]:
print(response["messages"][-1].content)

Hello! How can I assist you today?


## Testing Agent with Tool Usage

Now test with a query that requires external information. The agent should:
1. Recognize it needs current/specific information about "agentic AI"
2. Call the Wikipedia tool to gather relevant information  
3. Use the retrieved information to formulate a comprehensive response

In [ ]:
from langchain_core.messages import HumanMessage

#First up, let's see how it responds when there's no need to call a tool:
response = agent_executor.invoke({"messages": [HumanMessage(content="what is agentic ai")]})
response["messages"]

[HumanMessage(content='what is agentic ai', additional_kwargs={}, response_metadata={}, id='e6980ca3-f184-490d-8f29-f2e9ed04b5f7'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_MhztVgSsKN4gXCOYZHii2qcv', 'function': {'arguments': '{"query":"agentic AI"}', 'name': 'wikipedia'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 86, 'total_tokens': 101, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_c4c155951e', 'id': 'chatcmpl-C9j8tajMRktrVvKeQ7ZYKuMMtS88d', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--7d85fe3c-00cd-4016-8f65-902f4ce36511-0', tool_calls=[{'name': 'wikipedia', 'args': {'query': 'agentic AI'}, 'id': 'call_MhztVgSsKN4gXCO

In [10]:
pd.DataFrame([msg.__dict__ for msg in response["messages"]])

,content,additional_kwargs,response_metadata,type,name,id,example,tool_calls,invalid_tool_calls,usage_metadata,tool_call_id,artifact,status
0,what is agentic ai,{},{},human,None,e6980ca3-f184-490d-8f29-f2e9ed04b5f7,False,NaN,NaN,NaN,NaN,NaN,NaN
1,,{'tool_calls': [{'id': 'call_MhztVgSsKN4gXCOYZ...,"{'token_usage': {'completion_tokens': 15, 'pro...",ai,None,run--7d85fe3c-00cd-4016-8f65-902f4ce36511-0,False,"[{'name': 'wikipedia', 'args': {'query': 'agen...",[],"{'input_tokens': 86, 'output_tokens': 15, 'tot...",NaN,NaN,NaN
2,Page: Agentic AI\nSummary: Agentic AI is a cla...,{},{},tool,wikipedia,298b8671-9df9-477d-8f27-b8b34bc350d7,NaN,NaN,NaN,NaN,call_MhztVgSsKN4gXCOYZHii2qcv,NaN,success
3,Agentic AI is a class of artificial intelligen...,{'refusal': None},"{'token_usage': {'completion_tokens': 38, 'pro...",ai,None,run--eee64f7c-51a2-4773-a6da-d22420196fe3-0,False,[],[],"{'input_tokens': 162, 'output_tokens': 38, 'to...",NaN,NaN,NaN


In [11]:
print(response["messages"][-1].content)

Agentic AI is a class of artificial intelligence that focuses on autonomous systems capable of making decisions and performing tasks without human intervention. These systems can independently respond to conditions to produce desired outcomes.
